# Building Med-RAG: A RAG based model for medical queries

A full RAG pipeline using **Llama Index** will be built. We will install an open-source LLM **Phi-3**, from Microsoft, to run locally on our Colab instance to do the generation. To augment the results the LLM will access ([medquad data](https://www.kaggle.com/datasets/jpmiller/layoutlm/data)) stored and embedded in **Chroma database** as a vector store/document store. Finally, **Ollama** will be used as an interface to interact with Phi-3 on our machine.

Note: The below-described project uses guidance from both the lecture resources as well as OpenAI model.

## Installation of libraries and packages

In [ ]:
# import basic libraries (takes around 4 minutes)
import numpy as np
import pandas as pd

# basic installations
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama_index.readers.web
!pip install llama-index-vector-stores-chroma
!pip install chromadb

!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.38#' | sh
!pip install llama-index-embeddings-huggingface llama-index-llms-ollama llama-index-vector-stores-chroma llama-index ipywidgets llama-index-llms-huggingface chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  U

## Setting up OLLAMA

In [ ]:
# reference from lecture_8
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

time.sleep(5)  # Makes Python wait for 5 seconds

!ollama run $OLLAMA_MODEL "Give me short summary of Warwick Business School."

phi:latest
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b... 

In [ ]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install llama_index.readers.web
!pip install llama-index-vector-stores-chroma
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SummaryIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

# Use the global variable (OLLAMA_MODEL) as our LLM
llm = Ollama(model=OLLAMA_MODEL, request_timeout=480.0)

# Reading the dataset

MedQuAD includes 47,457 medical question-answer pairs created from 12 NIH websites (e.g. cancer.gov, niddk.nih.gov, GARD, MedlinePlus Health Topics). The collection covers 37 question types (e.g. Treatment, Diagnosis, Side Effects) associated with diseases, drugs and other medical entities such as tests.

For simplicity purposes, the below built Med-RAG contains 555 records of commonly known diseases and disorders.

In [ ]:
# Load the dataset
df = pd.read_excel("medquad_rag.xlsx", sheet_name='main')

# Explore the dataset
print(df.head())
print(df.info())

# Cleaning the Data

# Remove duplicates
df = df.drop_duplicates()

# Handle missing values - as 'questions' and 'answers' are essential columns
df = df.dropna(subset=['question', 'answer'])


                                            question  \
0                  What is (are) Anxiety Disorders ?   
1       What are the symptoms of Anxiety Disorders ?   
2                How to diagnose Anxiety Disorders ?   
3  what research (or clinical trials) is being do...   
4  How many people are affected by Anxiety Disord...   

                                              answer           source  \
0  Occasional anxiety is a normal part of life. Y...  NIHSeniorHealth   
1  Excessive, Irrational Fear Each anxiety disord...  NIHSeniorHealth   
2  Anxiety disorders sometimes run in families, b...  NIHSeniorHealth   
3  Clinical trials are part of clinical research ...  NIHSeniorHealth   
4  Studies estimate that anxiety disorders affect...  NIHSeniorHealth   

          focus_area  
0  Anxiety Disorders  
1  Anxiety Disorders  
2  Anxiety Disorders  
3  Anxiety Disorders  
4  Anxiety Disorders  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (

# Chunking

In [ ]:
import spacy

# Download the SpaCy model
!python -m spacy download en_core_web_md

from transformers import AutoTokenizer, AutoModel
import torch
from llama_index.core import VectorStoreIndex, StorageContext, Document
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Load a SpaCy model
nlp = spacy.load("en_core_web_md")  # This model includes tagger, parser, named entity-recognition

# Function to perform semantic chunking and return chunks as texts
def semantic_chunking(text):
    doc = nlp(text)
    chunks = [chunk.text for chunk in doc.noun_chunks]  # Extract noun chunks
    return chunks

# Apply semantic chunking to the 'answer' column
chunked_data = df['answer'].apply(semantic_chunking)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 968.4 kB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.4 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Word Embedding

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
from llama_index.core import Document
import numpy as np

# Initialize the SentenceTransformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to create embeddings
def get_embeddings(texts):
    # Generate embeddings
    return model.encode(texts, convert_to_tensor=False)

# Get embeddings
embeddings = get_embeddings(chunked_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
documents = []
for i, (text, emb) in enumerate(zip(chunked_data, embeddings)):
    text_str = ' '.join(text) if isinstance(text, list) else str(text)
    # Convert numpy array to list
    emb_list = emb.tolist()
    # Create a document dictionary with text and embedding
    doc = {'doc_id': str(i), 'embedding': emb_list, 'text' : text_str}
    documents.append(doc)

# Print out a complete document dictionary to see its structure
if documents:
    print("Document:", documents[0])

Document: {'doc_id': '0', 'embedding': [0.10082398355007172, -0.008585035800933838, 0.06003247946500778, 0.12660792469978333, 0.021252810955047607, -0.03430977836251259, 0.08193982392549515, -0.010186100378632545, 0.062495991587638855, -0.10758744925260544, -0.06524046510457993, 0.032645173370838165, -0.00532910879701376, -0.048825304955244064, -0.014341382309794426, 0.035654615610837936, -0.05771520361304283, -0.03508003428578377, -0.04539822041988373, 0.1283159852027893, -0.18706107139587402, 0.03182431682944298, -0.05984170734882355, 0.016106121242046356, -0.049249980598688126, 0.04897737875580788, 0.019220614805817604, -0.05634883791208267, 0.06348636001348495, 0.03382648155093193, 0.03620965778827667, 0.015961868688464165, 0.006901115644723177, -0.004056353121995926, 0.06430841982364655, -0.02100100927054882, -0.006186324171721935, 0.05927230790257454, -0.009720386005938053, -0.002994736423715949, 0.008503634482622147, -0.0333213210105896, 0.024246474727988243, -0.0033286453690379

# Creating Vector Database and Integrating Embeddings

In [ ]:
# Convert dictionaries to Document objects making it suitable for integrating in vector database
documents_list = [
    Document(doc_id=doc["doc_id"], text=doc["text"], embedding=doc["embedding"]) for doc in documents
]

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from chromadb import PersistentClient
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

# Create client "db" and a database ("chroma_db")
db = PersistentClient(path="./chroma_db")

# Create a collection
chroma_collection = db.create_collection("med_rag_collection")

if documents:
    embedding_dim = len(documents[0]["embedding"])

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection, embedding_dim=embedding_dim)

# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index with the pre-embedded documents
index = VectorStoreIndex.from_documents(
    documents=documents_list,
    storage_context=storage_context
)

# Print the name of the collection
print(f'Created Collection : {chroma_collection.name}')

Created Collection : med_rag_collection


In [ ]:
# Print the metadata
print(chroma_collection)

name='med_rag_collection' id=UUID('b6511338-14d9-4920-97d5-112034a54ce3') metadata=None tenant='default_tenant' database='default_database'


# Query Modes

In [ ]:
# running query_engine in various modes to compare and see response generated

**Compact**

In [ ]:
query_engine = index.as_query_engine(response_mode="compact")
response = query_engine.query("What is cancer?")
response.response

' According to the given context information, cancer refers to a group of diseases characterized by the uncontrolled growth and spread of abnormal cells in the body. These abnormal cells can form masses or tumors that can invade nearby tissues and spread to other parts of the body through metastasis. There are more than 100 different types of cancer, with each type affecting specific organs or tissues. The treatment for cancer varies depending on the type, stage, and location of the tumor, but it often involves a combination of surgery, radiation therapy, chemotherapy, hormone therapy, biologic therapy, and stem cell transplantation. Symptoms of cancer can vary depending on the type and extent of the disease.\n'

**Refine**

In [ ]:
query_engine = index.as_query_engine(response_mode="refine")
response = query_engine.query("What is cancer?")
response.response

" Based on your query about what cancer is, here's a more detailed explanation: Cancer is a group of diseases characterized by the uncontrolled growth and spread of abnormal cells in the body. These cells can invade nearby tissues and organs and can even travel through the bloodstream or lymphatic system to other parts of the body, causing further damage. In cancer, cells divide and grow uncontrollably, forming tumors that can interfere with normal bodily functions. Cancer is a complex disease that affects different parts of the body in different ways, and its causes are often not fully understood. However, factors such as genetics, lifestyle choices, environmental exposures, and infections have been linked to an increased risk of developing cancer.\n"

**Tree Summarize**

In [ ]:
query_engine = index.as_query_engine(response_mode="tree_summarize")
response = query_engine.query("How can we cure depression?")
response.response

" Depression is a serious medical illness with various causes including genetics, biology, environment, and psychology. While there's no single solution that works for every individual, new antidepressant medications such as selective serotonin reuptake inhibitors (SSRIs) have been found to be effective in managing the condition. These medications work by increasing levels of neurotransmitters like norepinephrine and dopamine, which are responsible for regulating moods. However, it's important to note that antidepressants should be taken under a doctor's supervision and as part of a comprehensive treatment plan that may also include talk therapy, lifestyle changes, and other interventions.\n"

# Prompt Template

With regards to above query mode outputs, **tree summarize** gives better and concise outputs and it captures nuanced relationships between medical concepts.

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Prompt Template
answer_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Setting up chat messages for the template
chat_messages = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="Direct and concise response required."
    ),
    ChatMessage(role=MessageRole.USER, content=answer_prompt_str),
]

# Creating the prompt template
answer_template = ChatPromptTemplate(chat_messages)

In [ ]:
# Query setup for a concise response

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("What causes nausea?")

print(response)

 Nausea is typically caused by an irritation or dysfunction in the digestive system, but it can also be triggered by other factors such as motion sickness, pregnancy, certain medications, anxiety, stress, or food poisoning. Other possible causes of nausea include infections, liver disease, and cancer. It's important to determine the underlying cause of nausea to effectively treat it.



In [ ]:
# Query setup for generative response

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("Can you list numbers from 1-20?")

print(response)

 Sure! Here are the numbers 1 through 20:
```
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
```



The outputs generated above shows that the model is capable of answering both medical queries and other general questions showing its capability as an LLM.

# Testing

In [ ]:
# Query setup for preventive measures

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("preventive measures for covid-19 ")

print(response)

 According to the given context information, there are several preventive measures that can be taken to reduce the risk of contracting COVID-19. These include:
1. Wearing a mask in public places where social distancing is difficult or impossible.
2. Practicing good hygiene, such as washing your hands frequently with soap and water for at least 20 seconds or using hand sanitizer.
3. Avoiding close contact with people who are sick.
4. Staying home when you are feeling ill.
5. Covering your mouth and nose with a tissue or your elbow when coughing or sneezing.
6. Cleaning frequently-touched surfaces, such as doorknobs and countertops, with disinfectant.
7. Avoiding large gatherings and staying informed about the latest guidelines from health authorities. 
It is important to note that these measures can help reduce the risk of contracting COVID-19, but they are not foolproof and should be used in conjunction with other safety practices, such as social distancing and wearing a mask in public

In [ ]:
# Query setup for first aid

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("A snake has bit me. What should I do?")

print(response)

 If you have been bitten by a snake, follow these steps:

1. Stay calm and try to keep still to avoid aggravating the venom.
2. Clean the wound with soap and water if possible, being careful not to disturb any of the venom or puncture wounds on your skin.
3. Apply pressure to the bite area using a clean cloth or bandage to slow down the spread of venom.
4. Seek medical attention immediately, even if you don't experience symptoms right away. It's important to get treated with antivenom as soon as possible to prevent serious complications.




In [ ]:
# Query setup for treatment identification

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("What are the treatment options for migraine?")

print(response)

 There are several treatment options available for migraine. Some of the most common ones include:

1. Over-the-counter pain relievers such as ibuprofen or aspirin can help alleviate mild to moderate symptoms.
2. Prescription medications like triptans, which work by constricting blood vessels in the head and neck, are also effective for treating migraines.
3. Preventive medications like beta-blockers, calcium channel blockers, and antidepressants may be prescribed to reduce the frequency of migraines or prevent them from occurring at all.
4. Alternative therapies such as biofeedback training, acupuncture, or massage can help alleviate symptoms in some cases.
5. Lifestyle changes such as stress reduction techniques, regular exercise, and a healthy diet can also play an important role in managing migraine symptoms.

It's important to work closely with a healthcare professional to determine the most effective treatment plan for each individual case of migraine.



In [ ]:
# Query setup for disease detection

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("I am a student who is under exam stress. There is a sudden weight gain with frequent bad moods. suggest remedy.")

print(response)

 Hello! Based on the context given, it seems that you may be experiencing symptoms of anxiety disorder. It's important to talk to your doctor about this and they can recommend treatments such as medication or therapy. In addition, practicing stress-reducing activities like exercise, meditation, and deep breathing techniques could also help alleviate some of these symptoms.



In [ ]:
# Query setup for medical suggestions

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("sudden change in food habits. increase in pimple and insomnia. suggest life style changes")

print(response)

 Thank you for using our service. Based on the given context, I can provide some suggestions for your lifestyle changes to improve both your sleep quality and skin health.
First, let's address the issue of insomnia. Insomnia is a common problem that can be caused by various factors such as stress, anxiety, caffeine intake, or irregular sleep schedule. To improve your sleep quality, it is important to establish a regular sleep routine and create a comfortable sleeping environment. This includes avoiding stimulating activities before bedtime, limiting the use of electronic devices, and keeping the bedroom dark and quiet.
As for the increase in pimples, it could be related to your diet or lifestyle habits. A poor diet high in processed foods and sugar can contribute to acne breakouts. It is recommended to follow a balanced diet that includes plenty of fruits, vegetables, whole grains, and lean proteins. You should also drink plenty of water to flush out toxins from your body and keep your

In [ ]:
# Query setup for medications and supplements

response = index.as_query_engine(
    response_mode="tree_summarize",
    text_qa_template=answer_template,
    llm=llm,
).query("suggest medications and supplements for sleep apnea")

print(response)

 Insomnia is a common sleep disorder that can cause trouble falling asleep or staying asleep. It's important to consult with a healthcare provider to get an accurate diagnosis and treatment plan. Based on your medical history, physical exam, and sleep study results, it may be recommended to use medications such as melatonin, valerian root, or herbal remedies like chamomile tea. Additionally, lifestyle changes such as regular exercise, avoiding caffeine and alcohol, and creating a relaxing bedtime routine can help improve the quality of your sleep. It's important to remember that any new medication should be used under the guidance of a healthcare provider.



# Chat based interaction

A chat based command-line interaction (developed using AI guidance) where the user can continuously interact and clarify medical queries to Med-RAG.

Given the limited scope of the model and resources available, it is important to precisely describe the query with relevant details for a proper response. The model might also provide additional examples and explanations at certain instances for better user understanding for the query.

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

print("\nPlease type your query in the input field to begin.\nEnter 'exit' to end the conversation.\n\n")

# Function to dynamically generate a chat prompt template based on the conversation history
def generate_prompt_template(conversation_history):
    messages = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content="Direct and concise response required."
        )
    ]
    messages.extend(conversation_history)
    return ChatPromptTemplate(messages)

# Function to handle chat interaction
def chat_session():
    print("Med-RAG: Hi, how can I assist you today?")
    conversation_history = []
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            print("Med-RAG: Goodbye!")
            break
        else:
            ask_question(user_input, conversation_history)

def ask_question(question, conversation_history):
    # Append user question to history
    conversation_history.append(ChatMessage(role=MessageRole.USER, content=question))

    # Generate a new prompt template with the updated conversation history
    current_prompt_template = generate_prompt_template(conversation_history)

    # Use the query engine with the new prompt template
    response = index.as_query_engine(
        response_mode="tree_summarize",
        text_qa_template=current_prompt_template,
        llm=llm,
    ).query(question)

    # Process the response
    response_text = response.text if hasattr(response, 'text') else str(response)

    # Print response and also add it to the conversation history
    print(f"Med-RAG: {response_text}")
    conversation_history.append(ChatMessage(role=MessageRole.SYSTEM, content=response_text))

# Run the chat session
if __name__ == "__main__":
    chat_session()



Please type your query in the input field to begin.
Enter 'exit' to end the conversation.


Med-RAG: Hi, how can I assist you today?
User: I have extreme anxiety. what should I do?
Med-RAG:  Extreme anxiety can be difficult to manage on your own, so it's important to seek help from a mental health professional who has experience treating anxiety disorders. They can provide you with guidance and support as you work through your anxiety symptoms. Additionally, some people find that cognitive-behavioral therapy (CBT) or other forms of talk therapy can be helpful in managing their anxiety. If medication is needed, your doctor may prescribe an anti-anxiety medication to help alleviate symptoms. It's important to keep in mind that everyone experiences anxiety differently, so it's important to work with a professional who can provide personalized treatment options tailored to your specific needs.

User: lifestyle changes for bloating
Med-RAG:  To help reduce bloating, it is recommended to ma